In [1]:
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference, selection_rate
import pickle
import json
import pandas as pd
import numpy as np

In [2]:
MODEL_PATH = '../2_models/'
MODEL_NAME = 'random_forest_model'

DATA_PATH = '../1_data/3_split/'

SENSITIVE_FEATURE = 'sex'

In [4]:
# Récupération du modèle
with open(MODEL_PATH + MODEL_NAME + '.pkl', "rb") as f:
    loaded_model = pickle.load(f)

# Récupération des données
X_test = pd.read_csv(DATA_PATH + 'X_test.csv', index_col=0)
y_test = pd.read_csv(DATA_PATH + 'y_test.csv', index_col=0)
S_test = X_test[SENSITIVE_FEATURE]
y_pred = loaded_model.predict(X_test)

dp = demographic_parity_difference(y_test, y_pred, sensitive_features=S_test)

eo = equalized_odds_difference(y_test, y_pred, sensitive_features=S_test)

group_0 = y_pred[S_test == 0]
group_1 = y_pred[S_test == 1]
rate_0 = np.mean(group_0)
rate_1 = np.mean(group_1)
di = min(rate_0, rate_1) / max(rate_0, rate_1)

with open(MODEL_PATH + MODEL_NAME + '.json', "r") as f:
    model_json = json.load(f)

model_json['metriques']['fairness'] = {
    'demographic_parity' : dp,
    'equalized_odds' : eo,
    'disparate_impact' : di
} 

with open(MODEL_PATH + MODEL_NAME + '.json', "w") as f:
    json.dump(model_json, f, indent=4)